In [ ]:
import random
import numpy as np
import math
import statistics

class Genetic_Algorithm:
    def __init__(self,gen_limit,goal=None,lowest_best=False,pop_limit=1000,kill_rate=0.5,reprod_rate=0.3,clone_rate=0.2):
        self.game = Game()
        
        self.goal = goal
        self.LOWEST_FITNESS_IS_BEST = lowest_best
        self.GENERATION_LIMIT = gen_limit
        self.POPULATION_LIMIT = pop_limit
        self.KILL_RATE = kill_rate
        self.REPRODUCTION_RATE = reprod_rate
        self.CLONE_PROB = clone_rate
        
        self.MIN_CIRCLES = 3
        self.MAX_CIRCLES = 3

        # MIN always greater than 1 to avoid errors (wider than radius when drawin)
        self.MIN_RADIUS = 20
        self.MAX_RADIUS = 20

        self.MIN_MASS = 20 
        self.MAX_MASS = 100

        self.MIN_FRICTION = 1
        self.MAX_FRICTION = 10

        self.X_MIN = 0
        self.X_MAX = 600
        self.Y_MIN = 200
        self.Y_MAX = 400

        self.SPRING_JOINT_PROB = 0.5
        self.PIN_JOINT_PROB = 0.5

        self.MIN_REST_LENGTH = 200
        self.MAX_REST_LENGTH = 200

        self.MIN_STIFFNESS = 200
        self.MAX_STIFFNESS = 200

        self.MIN_DAMPNESS = 50
        self.MAX_DAMPNESS = 50

        self.MIN_RELAXED_DURATION = 50*0.1
        self.MAX_RELAXED_DURATION = 50*3

        self.MIN_STRAINED_DURATION = 50*0.1
        self.MAX_STRAINED_DURATION = 50*3
    
    # ------------have to be redefined for every problem/game () ---------------------------   
    def generate_child(PRINT_DEBUG=False):
        
        numberOfCircles = random.randint(self.MIN_CIRCLES,self.MAX_CIRCLES)
        if PRINT_DEBUG:
            print("Number of Circles: ", numberOfCircles)
        radius_list = [random.randint(self.MIN_RADIUS,self.MAX_RADIUS) for x in range(numberOfCircles)]
        if PRINT_DEBUG:
            print("Radius List: ", radius_list)
        mass_list = [self.MIN_MASS + int((self.MAX_MASS-self.MIN_MASS)*random.random()) for x in range(numberOfCircles)]
        if PRINT_DEBUG:
            print("Mass List: ", mass_list)
        friction_list = [self.MIN_FRICTION + int((self.MAX_FRICTION-self.MIN_FRICTION)*random.random()) for x in range(numberOfCircles)]
        if PRINT_DEBUG:
            print("Friction List: ", friction_list)
        position_list = [Vec2d(self.X_MIN + (self.X_MAX-self.X_MIN) * random.random(), self.Y_MIN + (self.Y_MAX-self.Y_MIN) * random.random()) for x in range(numberOfCircles)]
        if PRINT_DEBUG:    
            print("Position List: ", position_list)

        nodes = [i for i in range(numberOfCircles)]
        
        while True:
            edges = []
            
            for pair in itertools.combinations(nodes, r=2):

                c1 = pair[0]
                c2 = pair[1]
                
                if random.random() < self.SPRING_JOINT_PROB
                    edges.extend([(c1,c2,{'joint_type': 0, 'length': None,'rest_length': None,'stiffness': None,
                                          'dampness': None,'relaxed_duration': None,'strained_duration': None}),
                                  (c2,c1,{'joint_type': 0, 'length': None,'rest_length': None,'stiffness': None,
                                          'dampness': None,'relaxed_duration': None,'strained_duration': None})])
                elif random.random() < self.PIN_JOINT_PROB:
                    edges.extend([(c1,c2,{'joint_type': 1, 'length': None,'rest_length': None,'stiffness': None,
                                          'dampness': None,'relaxed_duration': None,'strained_duration': None}),
                                  (c2,c1,{'joint_type': 1, 'length': None,'rest_length': None,'stiffness': None,
                                          'dampness': None,'relaxed_duration': None,'strained_duration': None})])
            
            g = nx.DiGraph()
            g.add_edges_from(edges)
            if nx.is_strongly_connected(g):
                break;
        
        for edge in edges:
            edge[2]['length'] = position_list[edge[0]].get_distance(position_list[edge[1]])
            edge[2]['rest_length'] = self.MIN_REST_LENGTH + (self.MAX_REST_LENGTH-self.MIN_REST_LENGTH)*random.random()
            edge[2]['stiffness'] = self.MIN_STIFFNESS + (self.MAX_STIFFNESS-self.MIN_STIFFNESS)*random.random()
            edge[2]['dampness'] = self.MIN_DAMPNESS + (self.MAX_DAMPNESS-self.MIN_DAMPNESS)*random.random()
            edge[2]['relaxed_duration'] = self.MIN_RELAXED_DURATION + int((self.MAX_RELAXED_DURATION-self.MIN_RELAXED_DURATION)*random.random())
            edge[2]['strained_duration'] = self.MIN_STRAINED_DURATION + int((self.MAX_STRAINED_DURATION-self.MIN_STRAINED_DURATION)*random.random())
            
        if PRINT_DEBUG:
            print("Number of joints: ", len(edges))
            
        return [numberOfCircles,radius_list,mass_list,friction_list,position_list,edges]
    
    def fitness_function(child):
        self.game.simulatePhysics(child)
        return game.distance
    
    def undo_border_crossing(lower_bound,upper_bound,value):
        if value < lower_bound:
            return lower_bound
        elif value > upper_bound:
            return upper_bound
        else:
            return value
    
    def mutate(child):
        
        # MIN_RADIUS+MIN_RADIUS_DIFF always greater than 1 to avoid errors (wider than radius when drawin)
        MIN_RADIUS_DIFF = -1
        MAX_RADIUS_DIFF = 1

        MIN_MASS_DIFF = -5
        MAX_MASS_DIFF = 5

        # MIN_FRICTION + MIN_FRICTION_DIFF always greater than 0 to avoid errors (no negative friction)
        MIN_FRICTION_DIFF = -1
        MAX_FRICTION_DIFF = 1
 
        MIN_X_DIFF = -5
        MAX_X_DIFF = 5
        MIN_Y_DIFF = -5
        MAX_Y_DIFF = 5

        MIN_REST_LENGTH_DIFF = 0
        MAX_REST_LENGTH_DIFF = 0

        MIN_STIFFNESS_DIFF = 0
        MAX_STIFFNESS_DIFF = 0

        MIN_DAMPNESS_DIFF = 0
        MAX_DAMPNESS_DIFF = 0

        MIN_RELAXED_DURATION_DIFF = -50*0.1
        MAX_RELAXED_DURATION_DIFF = 50*0.1

        MIN_STRAINED_DURATION_DIFF = -50*0.1
        MAX_STRAINED_DURATION_DIFF = 50*0.1
        
        radius_list = child[1]
        child[1] = [undo_border_crossing(self.MIN_RADIUS,self.MAX_RADIUS,radius+(MIN_RADIUS_DIFF + (MAX_RADIUS_DIFF-MIN_RADIUS_DIFF)*random.random())) for radius in radius_list]
        
        mass_list = child[2]
        child[2] = [undo_border_crossing(self.MIN_MASS,self.MAX_MASS,mass+(MIN_MASS_DIFF + (MAX_MASS_DIFF-MIN_MASS_DIFF)*random.random())) for mass in mass_list]
        
        friction_list = child[3]
        child[3] = [undo_border_crossing(self.MIN_FRICTION,self.MAX_FRICTION,friction+(MIN_FRICTION_DIFF + (MAX_FRICTION_DIFF-MIN_FRICTION_DIFF)*random.random())) for friction in friction_list]
        
        position_list = child[4]
        child[4] = [Vec2d(undo_border_crossing(self.X_MIN,self.X_MAX,position.x+(MIN_X_DIFF + (MAX_X_DIFF-MIN_X_DIFF)*random.random())),undo_border_crossing(self.Y_MIN,self.Y_MAX,position.y+(MIN_Y_DIFF + (MAX_Y_DIFF-MIN_Y_DIFF)*random.random()))) for position in position_list]
        
        edges = child[5]
        for edge in edges:
            edge[2]['joint_type'] = random.randint(0,1)
            edge[2]['length'] = child[4][edge[0]].get_distance(position_list[edge[1]])
            edge[2]['rest_length'] = undo_border_crossing(self.MIN_REST_LENGTH,self.MAX_REST_LENGTH,edge[2]['rest_length']+(MIN_REST_LENGTH_DIFF + (MAX_REST_LENGTH_DIFF-MIN_REST_LENGTH_DIFF)*random.random()))
            edge[2]['stiffness'] = undo_border_crossing(self.MIN_STIFFNESS,self.MAX_STIFFNESS,edge[2]['stiffness']+(MIN_STIFFNESS_DIFF + (MAX_STIFFNESS_DIFF-MIN_STIFFNESS_DIFF)*random.random()))
            edge[2]['dampness'] = undo_border_crossing(self.MIN_DAMPNESS,self.MAX_DAMPNESS,edge[2]['dampness']+(MIN_DAMPNESS_DIFF + (MAX_DAMPNESS_DIFF-MIN_DAMPNESS_DIFF)*random.random()))
            edge[2]['relaxed_duration'] = undo_border_crossing(self.MIN_RELAXED_DURATION,self.MAX_RELAXED_DURATION,edge[2]['relaxed_duration']+(MIN_RELAXED_DURATION_DIFF + (MAX_RELAXED_DURATION_DIFF-MIN_RELAXED_DURATION_DIFF)*random.random()))
            edge[2]['strained_duration'] = undo_border_crossing(self.MIN_STRAINED_DURATION,self.MAX_STRAINED_DURATION,edge[2]['strained_duration']+(MIN_STRAINED_DURATION_DIFF + (MAX_STRAINED_DURATION_DIFF-MIN_STRAINED_DURATION_DIFF)*random.random()))
            
        return None
    
    def breed_population(population,breed_count,clone_prob):
        #print("---Breeding starts now:---------")
        prob = get_survival_probability(population,goal)
        indeces = [i for i in range(len(population))]
        for i in range(breed_count):
            parent_indeces = np.random.choice(indeces,1,p=prob,replace=False)
            parent1 = population[parent_indeces[0]]
            child = copy.deepcopy(parent1)
            if random.random() > clone_prob:
                mutate(child)
            #print("Mutated Child is: ", convert_to_string(child))
            population.append(child)
        #print("------Breeding has ended-------")
    
    # ---------------------------------------- ---------------------------
    
    def createRandomMachine(space, child=[]):
        
        if child == []:
            child = generate_child()

        numberOfCircles = child[0]
        radius_list = child[1]
        mass_list = child[2]
        friction_list = child[3]
        position_list = child[4]
        edges = child[5]

        circles = [Circle(space,position_list[i],radius_list[i],mass_list[i],friction_list[i]) for i in range(numberOfCircles)]

        for edge in edges:
            index1 = edge[0]
            index2 = edge[1]
            c1 = circles[index1]
            c2 = circles[index2]

            if edge[2]['joint_type'] == 0:
                rest_length = edge[2]['rest_length']
                stiffness = edge[2]['stiffness']
                dampness = edge[2]['dampness']
                relaxed_duration = edge[2]['relaxed_duration']
                strained_duration = edge[2]['strained_duration']
                spring_joints.append(SpringJoint(space,c1.body, c2.body, rest_length, stiffness,dampness, relaxed_duration,
                                                 strained_duration,i1=index1,i2=index2))
            else:    
                pin_joints.append(PinJoint(space,c1.body, c2.body,i1=index1,i2=index2))

        return circles,spring_joints,pin_joints

    def generate_population(population_size):  
        return [generate_child() for pop in range(population_size)]
    
    def kill_population(population,survivor_count):
        prob = get_survival_probability(population)
        rest_population_indeces = np.random.choice([i for i in range(len(population))], survivor_count, p=prob, replace=False)  
        rest_population = [population[rest_population_indeces[i]] for i in range(len(rest_population_indeces))]
        return rest_population

    def get_survival_probability(population):
        fitness_list = [fitness_function(individuals) for individuals in population]

        mean = statistics.mean(fitness_list)
        stdev = statistics.stdev(fitness_list)
        normalized_fitness_list = [(fitness-mean)/stdev for fitness in fitness_list]

        if self.lowest_fitness_is_best:
            changed_order_fitness_list = [np.e**(-fitness) for fitness in normalized_fitness_list]
        else:
            changed_order_fitness_list = [np.e**(fitness) for fitness in normalized_fitness_list]

        fitness_sum = sum(changed_order_fitness_list)
        prob_list = [fitness/fitness_sum for fitness in changed_order_fitness_list]

        return prob_list

    def start(self):

        survivor_count = int(self.POPULATION_LIMIT * (1-self.KILL_RATE))
        breed_count = int(self.POPULATION_LIMIT * self.REPRODUCTION_RATE)

        population = generate_population(self.POPULATION_LIMIT)

        for generations_count in range(self.GENERATION_LIMIT):
            sorted_fitness,sorted_population = zip(*sorted(zip([fitness_function(child) for child in population], population)))
            #print("This is the current sorted population: ", [convert_to_string(child) for child in sorted_population])
            #print("Its fitness list is: ", sorted_fitness)
            #print("This is the survival probability: ", get_survival_probability(sorted_population,goal))
            rest_population = kill_population(population,survivor_count)
            #print("After killing, this is the rest population: ", [convert_to_string(child) for child in rest_population])
            breed_population(rest_population,breed_count,self.CLONE_PROB)
            #print("After breeding, this is the rest population", [convert_to_string(child) for child in rest_population])
            rest_population.extend(generate_population(self.POPULATION_LIMIT-len(rest_population)))
            #print("To get to pop limit, rest has been added: ", [convert_to_string(child) for child in rest_population])
            population = rest_population
            print(generations_count+1,": ", list(sorted_fitness)[-1])
        
        return list(sorted_fitness)[-1], generations_count+1

print("Start:")
g = Genetic_Algorithm(1)
[best_distance, final_count] = g.start()
print("The best child has distance \"", final_sentence, "\" and was found after ", final_count, " generations.")
